`quote` makes a `begin` block, which is an `Expr`.
An `Expr` stores expressions, evaluation of which returns the last `return` object.

Just as `"$va"` interpolates `va` in the process of making a `String`, `$` interpolates variables in the process of making an `Expr`.

In [ ]:
va = 8;

In [ ]:
# This `Expr` interpolates `va`, which has been defined to be 8.
qu = quote
    1 + 2
    3 * $va
end
println(qu, " IS A ", typeof(qu), " AND EVALUATES TO ", eval(qu))

In [ ]:
# This `Expr` plans to use `va`, which will be defined to be 8.
qu = quote
    1 + 2
    3 * va
end
println(qu, " IS A ", typeof(qu), " AND EVALUATES TO ", eval(qu))

In [ ]:
try
    # This `Expr` interpolates `no`, which has not been defined.
    quote
        1 + 2
        3 * $no
    end
catch er
    println(er)
end

In [ ]:
# This `Expr` plans to use `no`, which will not be defined.
qu = quote
    1 + 2
    3 * no
end
try
    println(qu, " IS A ", typeof(qu), " AND EVALUATES TO ", eval(qu))
catch er
    println(er)
end

`macro`s make expressions at parse time.
A `macro` takes expressions, symbols, andor literals, makes one of them, and replaces itself with it, which is then immediately evaluated.

In [ ]:
macro ma(ex)
    # This `Expr` is `ex`, which is interpolated.
    ex
end
println(
    (@macroexpand @ma 1 + 2),
    " BECOMES A ",
    typeof(@macroexpand @ma 1 + 2),
    " THAT EVALUATES TO ",
    (@ma 1 + 2),
    " OF TYPE ",
    typeof(@ma 1 + 2),
)

In [ ]:
macro ma(ex)
    # This `Expr` contains `ex`, which will be interpolated.
    quote
        ex
    end
end
try
    println(
        (@macroexpand @ma 1 + 2),
        " BECOMES A ",
        typeof(@macroexpand @ma 1 + 2),
        " THAT EVALUATES TO ",
        (@ma 1 + 2),
        " OF TYPE ",
        typeof(@ma 1 + 2),
    )
catch er
    println(er)
end

In [ ]:
macro ma(ex)
    # This `Expr` contains `ex`, which is interpolated.
    quote
        $ex
    end
end
println(
    (@macroexpand @ma 1 + 2),
    " BECOMES A ",
    typeof(@macroexpand @ma 1 + 2),
    " THAT EVALUATES TO ",
    (@ma 1 + 2),
    " OF TYPE ",
    typeof(@ma 1 + 2),
)

In [ ]:
macro ma(ex)
    # This is not an `Expr` but just a `String`.
    "ex"
end
println(
    (@macroexpand @ma 1 + 2),
    " BECOMES ",
    typeof(@macroexpand @ma 1 + 2),
    " THAT EVALUATES TO ",
    (@ma 1 + 2),
    " OF TYPE ",
    typeof(@ma 1 + 2),
)

In [ ]:
macro ma(ex)
    # This is a `String` that has interpolated `ex`.
    "$ex"
end
println(
    (@macroexpand @ma 1 + 2),
    " REPLACES ITSELF WITH ",
    typeof(@macroexpand @ma 1 + 2),
    " THAT EVALUATES TO ",
    (@ma 1 + 2),
    " OF TYPE ",
    typeof(@ma 1 + 2),
)

In [ ]:
macro ma(ex)
    # This `Expr` contains a `String` that plans to interpolate `ex`.
    quote
        "$ex"
    end
end
try
    println(
        (@macroexpand @ma 1 + 2),
        " BECOMES A ",
        typeof(@macroexpand @ma 1 + 2),
        " THAT EVALUATES TO ",
        (@ma 1 + 2),
        " OF TYPE ",
        typeof(@ma 1 + 2),
    )
catch er
    println(er)
end

In [ ]:
macro ma(ex)
    # This `Expr` contains a `String` that has interpolated `ex`.
    quote
        "$($ex)"
    end
end
println(
    (@macroexpand @ma 1 + 2),
    " BECOMES A ",
    typeof(@macroexpand @ma 1 + 2),
    " THAT EVALUATES TO ",
    (@ma 1 + 2),
    " OF TYPE ",
    typeof(@ma 1 + 2),
)